In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# parameters for algorithm and wealth
# Wt starting parameters
# Total lifespan to use portfolio
T = 5
# timestep
dt=1/252
# total number of days in the lifespan
total_days = int(T/dt)
# original investment
W_0 = 50000

# mean drift of risky investment
mu = 0.1
# risk free rate of return
r= 0.02
# volatility of risky investment
sigma = 0.2
# level of risk aversion
gamma = 4
# utility discount rate
rho = 0.05
# bequest
e = 0.01

# formula for calculating how the wealth process develops every timestep
def get_dW(self,W, pi, mu, r, c,dt, sigma):
    dZ = np.random.normal()*(np.sqrt(dt))
    drift = W*(pi*(mu-r)+r-c)
    dw = drift*dt + pi*sigma*W*dZ
    return dw

def get_reward(u, gamma):
    if gamma != 1:
        reward = (u**(1-gamma))/(1-gamma)
    else:
        reward = np.log(u)
        
    reward*=np.exp(-rho*t*dt)

    return reward

def mertons_frac(mu,r,sigma,gamma):
    top = mu-r
    bottom = gamma*sigma**2
    frac = top/bottom
    return frac

def get_v(rho, gamma, pi, mu, r):
    v = (rho - (1-gamma)*(pow(mu-r,2)/(2*gamma*pow(sigma,2))+r))/gamma
    return v


def cW_T(v,T,t):
    if v!= 0:
        c = v/(1+(v*e-1)*np.exp(-v*(T-t/252)*gamma))
    else:
        c = 1/(T-t/252 + e)
    return c

In [ ]:
# Agent parameters

class Merton:
        def __init__(self):
            # vector of time statespace  with 
            # time as a percentage of time left in episode
            self.time_states = [x/100 for x in range(101)]
            # vector of wealth statespace with different 
            # defined as different thresholds for amount of total wealth 
            # from 0 <= x_n < x_n+1 < 100000 : x = 10000*n: n = 0,1,2,...
            self.wealth_states = [y*10**4 for y in range(10)]
            


# number of episodes 
num_episodes = 50000

# records
ep_rewards = []
total_consumed  = []
agent = Merton()
c_values = []
pi = mertons_frac(mu,r,sigma,gamma)
if pi>1:
    pi = 1
v= get_v(rho, gamma, pi, mu, r)
final_wealth = []


# # simulating
for episode in range(num_episodes):
    episode_consumption = []
    if (episode+1)%100 ==0:
        print(episode+1)
    # reset environment
    # episode rewards 
    G = 0
    # starting wealth
    wealth = W_0
    for t in range(total_days):
        c = cW_T(v,T,t)
        c_values.append(c)
        # update wealth
        dW = get_dW(wealth,t,pi,mu,r,c,dt,sigma)
        # print(dW,c)
        wealth += dW
        episode_consumption.append(c*wealth*dt)
        # reward
        reward = get_reward(c*wealth*dt,gamma)
        if (t+1)== total_days: reward+= get_reward(wealth,gamma)
        G += reward
       
    ep_rewards.append(G)
    final_wealth.append(wealth)
    total_consumed.append(sum(episode_consumption))
    
tally = "Total money consumed: {:.2f}\n\
Total money leftover: {:.2f}"

MoneyC = np.mean(total_consumed)
MoneyL = np.mean(final_wealth)
print(tally.format(MoneyC,MoneyL))

In [ ]:
#  Plot results

mean_consumption = [np.mean(episode_consumption[n-30:n]) if n > 30 else np.mean(episode_consumption[:n]) 
               for n in range(1, len(episode_consumption))] 

x = np.linspace(0,1,len(mean_consumption))
plt.figure(figsize=(12,8))
plt.plot(x,mean_consumption)
plt.title('Consumption Over Time')
plt.xlabel('time')
plt.ylabel('consumption')
plt.savefig('Merton Consumption.png')
plt.show()   